<a href="https://colab.research.google.com/github/digwit678/Can-Language-Models-Follow-Discussions/blob/main/WANDB_API_Result_Data_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb

In [ ]:
import wandb
import scipy.stats as stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [ ]:
from google.colab import drive
GDRIVE_BASE_PATH = "/content/drive/MyDrive/Can Language Models Follow Discussions?/"
drive.mount('/content/drive')

Mounted at /content/drive


# Authenticate with W&B

In [ ]:
WANDB_API_KEY = "13fa8ba0883264fc7dfc29662e14d75902a76881"
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: ERROR API key must be 40 characters long, yours was 13
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
WANDB_USER_NAME = "digwit"
TIMEOUT = 60  # Increase the API timeout to 60 seconds
#wandb.login()
api = wandb.Api(timeout=TIMEOUT)

# CONSTANTS

In [ ]:
SAVING_PATH = 'probing_results/wandb_data_1.csv'

In [ ]:
PROJECT_NAMES = ["1_4_light_probes", "4_different_depths", "3_reactiveness", "5_claim_path_density", "1_opposing_affirming_claims", "2_Sequential-Claims-Hard"]
METRICS = ["full test f1", "full test acc"]
CONTROL_TASKS = ["RANDOMIZATION", "NONE", "PERMUTATION"]
MODELS = ['microsoft/deberta-v3-base', 'facebook/bart-base', 'gpt2', 'albert-base-v2', 'bert-base-uncased']
SEEDS = [0, 1, 2, 3, 4]
FOLDS = [0, 1, 2, 3]

# Create Analysis Dataframe

In [ ]:
def fetch_initial_data(api, user_name, project_names, metrics, control_tasks):
    data = []
    for project_name in project_names:
        runs = api.runs(f"{user_name}/{project_name}")
        for run in runs:
            if run.state == "finished":
                run_config = run.config
                model_name = run_config.get("model_name", "Unknown")
                seed = run_config.get("seed", -1)
                fold = run_config.get("fold", -1)
                control_task_type = run_config.get("control_task_type", "Unknown")

                for metric in metrics:
                    if metric in run.history().columns:
                        metric_value = run.history()[metric].mean()  # Assuming mean for aggregation
                        data.append({
                            "project": project_name,
                            "model": model_name,
                            "seed": seed,
                            "fold": fold,
                            "control_task": control_task_type,
                            "metric": metric,
                            "value": metric_value
                        })
    return data

# Fetch data
initial_data = fetch_initial_data(api, WANDB_USER_NAME , PROJECT_NAMES, METRICS, CONTROL_TASKS)

# Convert to DataFrame and save
initial_df = pd.DataFrame(initial_data)
initial_df.to_csv(f'{GDRIVE_BASE_PATH}/{SAVING_PATH}', index=False)

print("Initial data saved.")

Initial data saved.


In [ ]:
initial_df.head()

,project,model,seed,fold,control_task,metric,value
0,1_4_light_probes,microsoft/deberta-v3-base,2,2,NONE,full test f1,0.793477
1,1_4_light_probes,microsoft/deberta-v3-base,2,2,NONE,full test acc,0.797944
2,1_4_light_probes,microsoft/deberta-v3-base,4,3,NONE,full test f1,0.790188
3,1_4_light_probes,microsoft/deberta-v3-base,4,3,NONE,full test acc,0.794550
4,1_4_light_probes,microsoft/deberta-v3-base,2,3,NONE,full test f1,0.790557


 # Check for missing runs / parameter configurations

In [ ]:
#todo refactor
def check_missing_data(data):
    missing_data = []
    for project in PROJECT_NAMES:
        for model in data['model'].unique():
            for seed in range(5):  # Seeds 0 to 4
                for fold in range(4):  # Folds 0 to 3
                    for control_task in CONTROL_TASKS:
                        condition = (data['project'] == project) & (data['model'] == model) & (data['seed'] == seed) & (data['fold'] == fold) & (data['control_task'] == control_task)
                        if not data[condition].shape[0] == len(METRICS):
                            missing_data.append((project, model, seed, fold, control_task))
    return missing_data

# Check for missing combinations in each project
missing_combinations = {}
total_missing_count = 0

for project in PROJECT_NAMES:
    missing = []
    for model in MODELS:
        for seed in SEEDS:
            for fold in FOLDS:
                for task in CONTROL_TASKS:
                    if not ((initial_df['project'] == project) & (initial_df['model'] == model) &
                            (initial_df['seed'] == seed) & (initial_df['fold'] == fold) &
                            (initial_df['control_task'] == task)).any():
                        missing.append((seed, model, fold, task.lower()))
    missing_combinations[project] = missing
    project_missing_count = len(missing)
    total_missing_count += project_missing_count
    print(f"Project '{project}' is missing {project_missing_count} combinations.")



In [ ]:
missing_combinations['4_different_depths']

In [ ]:
missing_combinations['5_claim_path_density']

In [ ]:
missing_combinations['3_reactiveness']

## Generate a single chained command to rerun missing runs in probing framework


In [ ]:
all_commands = []
for project, missing in missing_combinations.items():
    for seed, model, fold, task in missing:
        cmd = f"python3 run_tasks.py --seeds {seed} --models {model} --control_tasks {task} --include_probes {project.split('_')[0]} --attach_topic True"
        all_commands.append(cmd)

chained_command = " && ".join(all_commands)
print(f"All projects combined are missing a total of {total_missing_count} combinations.")
print(f"Chained command to rerun all missing combinations:\n{chained_command}")


All projects combined are missing a total of 167 combinations.
Chained command to rerun all missing combinations:
python3 run_tasks.py --seeds 1 --models facebook/bart-base --control_tasks RANDOMIZATION --include_probes 4 --attach_topic True && python3 run_tasks.py --seeds 3 --models facebook/bart-base --control_tasks RANDOMIZATION --include_probes 4 --attach_topic True && python3 run_tasks.py --seeds 4 --models facebook/bart-base --control_tasks RANDOMIZATION --include_probes 4 --attach_topic True && python3 run_tasks.py --seeds 4 --models facebook/bart-base --control_tasks RANDOMIZATION --include_probes 4 --attach_topic True && python3 run_tasks.py --seeds 3 --models bert-base-uncased --control_tasks RANDOMIZATION --include_probes 4 --attach_topic True && python3 run_tasks.py --seeds 0 --models microsoft/deberta-v3-base --control_tasks RANDOMIZATION --include_probes 3 --attach_topic True && python3 run_tasks.py --seeds 0 --models microsoft/deberta-v3-base --control_tasks NONE --inclu

# Append Data to Analysis Dataframe

In [ ]:
saving_path_aggregated = "probing_results/updated_aggregated_mann_whitney_u_test_results.csv"
saving_path_runs = "probing_results/all_run_level_mann_whitney_u_test_results.csv"

In [ ]:
def append_to_csv(file_path, new_data):
    """
    Appends new data to an existing CSV file.

    :param file_path: String, the path to the CSV file.
    :param new_data: DataFrame, the new data to append.
    """
    # Load existing data from CSV
    try:
        existing_data = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"No existing file found at {file_path}. A new file will be created.")
        existing_data = pd.DataFrame()

    # Append new data
    updated_data = pd.concat([existing_data, new_data], ignore_index=True)

    # Save updated data to CSV
    updated_data.to_csv(file_path, index=False)
    print(f"Data successfully appended to {file_path}.")

# File paths for existing CSV files


aggregated_csv_path = f'{GDRIVE_BASE_PATH}/{saving_path_aggregated}'
run_level_csv_path = f'{GDRIVE_BASE_PATH}/{saving_path_runs}'

# Example usage:
# Assuming 'new_aggregated_data' and 'new_run_level_data' are the new DataFrames to append
append_to_csv(aggregated_csv_path, new_aggregated_data)
append_to_csv(run_level_csv_path, new_run_level_data)


# Save Dataframe

In [ ]:
#initial_df = pd.read_csv(f'{GDRIVE_BASE_PATH}/{SAVING_PATH}')

In [ ]:
initial_df.head()

,project,model,seed,fold,control_task,metric,value
0,1_4_light_probes,microsoft/deberta-v3-base,2,2,NONE,full test f1,0.793477
1,1_4_light_probes,microsoft/deberta-v3-base,2,2,NONE,full test acc,0.797944
2,1_4_light_probes,microsoft/deberta-v3-base,4,3,NONE,full test f1,0.790188
3,1_4_light_probes,microsoft/deberta-v3-base,4,3,NONE,full test acc,0.794550
4,1_4_light_probes,microsoft/deberta-v3-base,2,3,NONE,full test f1,0.790557


In [ ]:
SAVING_PATH_2 = 'probing_results/wandb_data_2.csv'